In [168]:
#The workflow for the chart is like this:
# First, edit the data in the main xlsx.
# Then, update the local xlsx, which should have a data query in it to the relevant table
# Then, save this as a csv (probably possible to eliminate this step)
# Then, run this script to create the graph. Edit parameters as needed

In [169]:
import plotly.io as pio
pio.renderers.default = "iframe"
# for some reason the native renderer doesn't work in anaconda/jupyter

In [170]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

dataframe_bubble = pd.read_csv("datasets/graph_table.csv", sep=";", decimal=",")

In [191]:
fig = px.scatter(dataframe_bubble, 
                 x="Intergenerational Solidarity Index", 
                 y="v2x_libdem", 
                 size="Population", 
                 color="Region",
                 hover_name="Country", 
                 size_max=60,
                )
fig.add_trace(go.Scatter
              (x=['0', '100']
               , y=[0.42, 0.42]
               , mode="lines"
               , name=""))
fig.add_trace(go.Scatter
              (x=['50', '50']
               , y=[-0.1, 1]
               , mode="lines"
               , name=""))
fig.update_traces(line_color='#456987')
   
fig.update_layout(showlegend=False,
                   width = 500,
                  height = 500,
                   margin=dict(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=0
                   ),
                autosize=False,
                plot_bgcolor="#eeeeee",
                yaxis = dict(
                scaleanchor = "x",
                scaleratio = 119,)#scale ratio usually something different
                 )
fig.update_xaxes(showgrid=False,
                zeroline=False,
                range=[0, 100],
                showticklabels=False,
                )
fig.update_yaxes(title_text='vDem Liberal Democracy',
                showgrid=False,
                zeroline=False,
                range=[0, 0.88], #remember to change this for other indexes
                showticklabels=False,)

fig.show()


In [192]:
#Use this to write an html if needed
# import plotly.io as pio
# pio.write_html(fig, file=’bubblechart.html’, auto_open=True)

In [193]:
import chart_studio

In [194]:
username = 'jaemikew' 
api_key = '3DMC84ZXuI11xkaPDFt3'

In [195]:
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

In [196]:
#this pushes it to plotly.com
import chart_studio.plotly as py
py.plot(fig, filename = 'bubblechart', auto_open=False)

'https://plotly.com/~jaemikew/9/'